Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import os
import datetime
import scipy

Loading Dataset

In [2]:
df = pd.read_csv(r"C:\Users\Vasumathi T\Drunk or Sober Prediction from Accelerometer Data\all_accelerometer_data_pids_13.csv")

In [3]:
df.head()

,time,pid,x,y,z
0,0,JB3156,0.0000,0.0000,0.0000
1,0,CC6740,0.0000,0.0000,0.0000
2,1493733882409,SA0297,0.0758,0.0273,-0.0102
3,1493733882455,SA0297,-0.0359,0.0794,0.0037
4,1493733882500,SA0297,-0.2427,-0.0861,-0.0163


In [5]:
df['pid'].value_counts()

CC6740    2374695
PC6771    2141701
HV0618    1876013
DK3500    1339622
BK7610    1225727
JB3156    1177749
SA0297     962901
SF3079     662949
MJ8002     631303
DC6359     591358
BU4707     447423
MC7070     318600
JR8022     307526
Name: pid, dtype: int64

In [6]:
df=df.drop(df.index[df["time"] == 0])

Splitting datapoints into time window of 60 seconds

In [7]:
def get_time_ignore_second(x):
    t = datetime.datetime.fromtimestamp(x/1000.0)
    t = t.replace(microsecond = 0)
    t = t.replace(second = int(t.second / 10))
    return t.timestamp()
df['t_win'] = df['time'].apply(get_time_ignore_second)

Feature Extraction

In [8]:
funcs = [('mean', np.mean), ('variance', np.var), ('median', np.median), ('min', np.min), ('max', np.max), ('rms', lambda x: np.sqrt(np.mean(np.square(x)))),  ('skew', scipy.stats.skew), ('Kurtiosis', scipy.stats.kurtosis)]
fft_funcs = [('variance', np.var), ('spectral_centroid_spread', lambda x: spectral_centroid_spread(x, 40)) ]
cols = ['x', 'y', 'z']

def spectral_centroid_spread(fft_magnitude, sampling_rate):
    ind = (np.arange(1, len(fft_magnitude) + 1)) * (sampling_rate / (2.0 * len(fft_magnitude)))
    eps = 0.00000001
    Xt = fft_magnitude.copy()
    Xt = Xt / Xt.max()
    NUM = np.sum(ind * Xt)
    DEN = np.sum(Xt) + eps
    centroid = (NUM / DEN)
    spread = np.sqrt(np.sum(((ind - centroid) ** 2) * Xt) / DEN)
    centroid = centroid / (sampling_rate / 2.0)
    return centroid

In [9]:
look_up_frames = dict()
for pid in df.pid.unique():
  rf = pd.DataFrame()
  grouped = df[df.pid == pid].groupby('t_win')
  #grouped.head()
  for col in cols:
    col_array = grouped[col].apply(np.array)
    col_fft = col_array.apply(scipy.fft.fft)
    for key, func in funcs:
      #print(pid, col, key)
      rf['_'.join([col, key])] = col_array.apply(func)
    for key, func in fft_funcs:
      #print(pid, col, key)
      rf['_'.join([col, 'FFT', key])] = col_fft.apply(func)
  rf.pid = pid
  look_up_frames[pid] = rf


F:\New folder\Lib\site-packages\pandas\core\apply.py:1174: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  mapped = lib.map_infer(
F:\New folder\Lib\site-packages\pandas\core\apply.py:1174: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  mapped = lib.map_infer(
F:\New folder\Lib\site-packages\pandas\core\apply.py:1174: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  mapped = lib.map_infer(
F:\New folder\Lib\site-packages\pandas\core\apply.py:1174: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  

Classification of datapoints into drunk/sober based on blood alcohol level

In [13]:
base_path = r"C:\Users\Vasumathi T\Drunk or Sober Prediction from Accelerometer Data\clean tac"
tac_dict = dict()
for file_path in os.listdir(base_path):
  resp_frame = pd.read_csv(base_path + '/'+ file_path)
  tac_dict[file_path.split('_')[0]] = resp_frame


def get_y(pid, t_value):
  ind = np.argmax(tac_dict[pid]['timestamp'] > t_value)
  if ind != 0:
    ind = ind - 1
  tac= (tac_dict[pid]).iloc[[ind]]['TAC_Reading'].values[0]
  if tac>0.08:
    return 1
  else:
    return 0

In [14]:
for pid in df.pid.unique():
  look_up_frames[pid]['t']=look_up_frames[pid].index
  look_up_frames[pid]['t']=look_up_frames[pid]['t'].astype(int)
  look_up_frames[pid]['pid']=pid
  look_up_frames[pid]['drunk']=0
  for i in range(len(look_up_frames[pid]['t'].index)):
   look_up_frames[pid]['drunk'].iloc[i]= get_y(pid,look_up_frames[pid]['t'].iloc[i])

C:\Users\Vasumathi T\AppData\Local\Temp\ipykernel_11580\877481042.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  look_up_frames[pid]['drunk'].iloc[i]= get_y(pid,look_up_frames[pid]['t'].iloc[i])
C:\Users\Vasumathi T\AppData\Local\Temp\ipykernel_11580\877481042.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  look_up_frames[pid]['drunk'].iloc[i]= get_y(pid,look_up_frames[pid]['t'].iloc[i])
C:\Users\Vasumathi T\AppData\Local\Temp\ipykernel_11580\877481042.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.or

Creation of final dataset

In [15]:
pids = list(df.pid.unique())
ff = pd.DataFrame()
ff = look_up_frames[pids[0]]
pids.pop(0)
for pid in pids:
  ff = ff.append(look_up_frames[pid], ignore_index=True)

C:\Users\Vasumathi T\AppData\Local\Temp\ipykernel_11580\1986353118.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ff = ff.append(look_up_frames[pid], ignore_index=True)


In [16]:
ff.to_csv(r"C:\Users\Vasumathi T\Drunk or Sober Prediction from Accelerometer Data\clean_data.csv")

In [17]:
ff.head()

,x_mean,x_variance,x_median,x_min,x_max,x_rms,x_skew,x_Kurtiosis,x_FFT_variance,x_FFT_spectral_centroid_spread,...,z_min,z_max,z_rms,z_skew,z_Kurtiosis,z_FFT_variance,z_FFT_spectral_centroid_spread,t,pid,drunk
0,-0.005482,0.001540,-0.0016,-0.2888,0.1672,0.039620,-3.152112,25.105315,0.256400,0.542149-0.615443j,...,-0.1177,0.1393,0.031382,-0.158666,5.648270,0.164358,0.265498+0.455852j,1493733844,SA0297,0
1,0.007375,0.004575,-0.0010,-0.2182,0.4063,0.068038,3.409275,17.744252,1.027660,-2.847666-4.559118j,...,-0.3146,0.7028,0.089431,3.240370,25.094149,1.775239,0.482896-0.414526j,1493733845,SA0297,0
2,0.000866,0.000292,-0.0002,-0.0107,0.2475,0.017108,13.737278,195.249854,0.064100,-0.578196-4.058292j,...,-0.1883,0.0062,0.013219,-14.224837,204.867043,0.038267,-0.528959+0.039652j,1493733900,SA0297,0
3,0.000034,0.000008,0.0000,-0.0124,0.0178,0.002774,0.823852,8.292259,0.001668,0.507541-0.262002j,...,-0.0079,0.0114,0.003066,2.803187,21.245055,0.002075,-0.351584+0.762266j,1493733901,SA0297,0
4,-0.000262,0.000008,-0.0005,-0.0140,0.0128,0.002892,0.490040,5.675891,0.001644,0.495195-0.089977j,...,-0.0083,0.0028,0.002425,-0.723952,1.651197,0.001292,1.319628+0.210628j,1493733902,SA0297,0


In [18]:
ff['pid'].value_counts()

CC6740    6935
PC6771    5377
HV0618    4741
SA0297    4403
JB3156    3404
DK3500    3372
BK7610    3089
SF3079    1671
MJ8002    1589
DC6359    1568
BU4707    1139
MC7070     804
JR8022     782
Name: pid, dtype: int64

In [19]:
list(ff.columns)

['x_mean',
 'x_variance',
 'x_median',
 'x_min',
 'x_max',
 'x_rms',
 'x_skew',
 'x_Kurtiosis',
 'x_FFT_variance',
 'x_FFT_spectral_centroid_spread',
 'y_mean',
 'y_variance',
 'y_median',
 'y_min',
 'y_max',
 'y_rms',
 'y_skew',
 'y_Kurtiosis',
 'y_FFT_variance',
 'y_FFT_spectral_centroid_spread',
 'z_mean',
 'z_variance',
 'z_median',
 'z_min',
 'z_max',
 'z_rms',
 'z_skew',
 'z_Kurtiosis',
 'z_FFT_variance',
 'z_FFT_spectral_centroid_spread',
 't',
 'pid',
 'drunk']